# Introduction to PyTorch

To extend the SGDClassifier from the previous notebook, there will be a jumpy from sklearn to PyTorch library.

## Instructions for All Labs
* Read each cell and implement the TODOs sequentially. The markdown/text cells also contain instructions which you need to follow to get the whole notebook working.
* Do not change the variable names unless the instructor allows you to.
* Some markdown cells contain questions.
  * For questions <span style="color:red;">colored in red</span>, you must submit your answers in the corresponding Assignment in the course page. Make sure that you enter your responses in the item with the matching question code. Answers that do not follow the prescribed format will automatically be marked wrong by the checker.
  * For questions <span style="color:green;">colored in green</span>, you don't have to submit your answers, but you must think about these questions as they will help enrich your understanding of the concepts covered in the labs.
* You are expected to search how to some functions work on the Internet or via the docs. 
* You may add new cells for "scrap work".
* The notebooks will undergo a "Restart and Run All" command, so make sure that your code is working properly.
* You may not reproduce this notebook or share them to anyone.

This notebook provides a short introduction to PyTorch, for a more in-depth introduction, refer to the official PyTorch [website](https://pytorch.org/tutorials/). 

PyTorch is an open-source machine learning library developed by Facebook, its primary usage is for implementing neural networks. But this does not mean that it is exclusive to neural networks. It can be used for general-purpose scientific computing, which is beyond the scope of this notebook.

You have to install PyTorch,

```shell
pip install torch
```

To take advantage of a GPU, it is recommended to upload this notebook to [Google Colab](https://colab.research.google.com/), but please do not use any code-generative features of the platform so you can actually learn the materials and concepts. [Lightning AI](https://lightning.ai/) is a nice alternative, but it usually requires account verification (within 24-48 hours). 

Note that Colab may have a PyTorch installation readily available.

## PyTorch Basics

We start with reviewing the basic concepts of PyTorch. Familiarity with `numpy` is recommended as it shares similar concepts with PyTorch. If you are not familiar with `numpy`, you may refer to their [official guide](https://numpy.org/devdocs/user/quickstart.html). 

Let's check the version of the package we have installed.

In [ ]:
import torch


torch.__version__

The `+cu124` indicates that the version installed is capable of using the Nvidia GPU in my machine.

To control the stochasticity of pseudorandom number generator, we should set the seed for the environment. For completeness sake, we can also set the seed for other libraries as some functions under the hood might be using them as well. There are also other functions to set the determinism of pseudorandom number generator in PyTorch.

In [ ]:
import random  # Python built-in library
import numpy as np

np.__version__  # for sanity checking

seed = 73
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
# benchmarking results to the library 
# computing the best algorithm to use for your hardware
torch.backends.cudnn.benchmark = False  

To initialize a tensor (a generalized mathematical object for structures, i.e. a scalar is a 0-d tensor, a vector is a 1-d tensor, and a matrix is a 2-d tensor), we can use the following function.

In [ ]:
initial_values = torch.empty(5, 3)

print(initial_values)

The above initialization does virtually the same thing as the following function,

In [ ]:
initial_values = torch.rand(5, 3)

print(initial_values)

To check the size or shape of the tensor,

In [ ]:
print(initial_values.size())
print(initial_values.shape)

To perform arithmetic operations in PyTorch, we may use the basic Python operations or the PyTorch-specific functions.

In [ ]:
print(initial_values + 1)
print()
print(torch.add(initial_values, 1))

# try torch.sub(), torch.mul(), torch.div()

There's also an in-place operation where we use the function with underscore suffix.

In [ ]:
print(initial_values.add(1))
print(initial_values)

print()

print(initial_values.add_(1))
print(initial_values)  # notice anything?

Slicing is the same as numpy,

In [ ]:
print(initial_values[:, 0])
print(initial_values[:, 1])
print(initial_values[:, 2])
print(initial_values[0, :])

Initialize one-tensor or zero-tensor,

In [ ]:
ones = torch.ones(5, 3)
zeros = torch.zeros(5, 3)

print(ones)
print(zeros)

Perform matrix transpose,

In [ ]:
# 5x3 matrix transposed is 3x5 matrix
# we transpose using the indices
print((initial_values.transpose(1, 0)))  

Use some transformation functions like logistic and softmax,

In [ ]:
print(torch.nn.functional.sigmoid(initial_values))
print(torch.nn.functional.softmax(initial_values))

Recall the formulas being,

$\sigma(z) = \dfrac{1}{1 + \exp(-z)}$

for logistic function (sigmoid). 

And the following for softmax,

$\sigma(z) = \dfrac{\exp(z)}{\sum \exp(z)}$

In [ ]:
print(
    1 / (1 + torch.exp(-initial_values))
)

print(
    torch.exp(initial_values) / torch.sum(torch.exp(initial_values), axis=1, keepdims=True)
)

However, performing softmax this way is numerically unstable. As the values become large, there is an overflow issue.

In [ ]:
samples = torch.Tensor([10, 2, 10_000, 4])

# axis=0 since this is only a vector
print(
    torch.exp(samples) / (torch.sum(torch.exp(samples), axis=0, keepdims=True))
)

How do we resolve this? Use max-value subtraction.

In [ ]:
samples = torch.Tensor([10, 2, 10_000, 4])
samples = torch.exp(samples - torch.max(samples))

# axis=0 since this is only a vector
print(
    torch.exp(samples) / (torch.sum(torch.exp(samples), axis=0, keepdims=True))
)

To perform some linear algebra operations,

In [ ]:
torch.matmul(
    initial_values, initial_values.transpose(1, 0)
)

In [ ]:
## inner product
torch.dot(samples, (samples - 1))

In [ ]:
## L2 norm
print(initial_values.norm(dim=1, p=2))
print(torch.linalg.norm(initial_values, dim=1, ord=2))
print(initial_values.pow(2).sum(dim=1).sqrt())

Stacking tensors,

In [ ]:
a = torch.randn(4)
b = torch.randn(4)
stacked_tensors = torch.stack([a, b])
print(stacked_tensors)
print(stacked_tensors.shape)

Flatten a tensor, e.g. from 2x4 tensor to a 8x1 tensor

In [ ]:
print(stacked_tensors.view(-1))
print(stacked_tensors.flatten())

Convert a numpy array to a tensor,

In [ ]:
samples = np.random.randn(2, 10)
print(samples)

samples = torch.from_numpy(samples)
print(samples)

`torch.from_numpy()` vs `torch.Tensor()`

* `torch.Tensor()` creates a new copy of the array, and automatically converts the array to float32
* `torch.from_numpy()` does not create a new copy, and preserves the dtype of the array

## Exercise

Given a list of three tensors, with each tensor representing a set of hypothetical model outputs, put them together so that each row would have all outputs for a given sample.

In [ ]:
out_1 = torch.Tensor(
    [
        [0.2562, 0.1650, 0.0918, 0.0045, 0.0175, 0.1096, 0.0831, 0.2002, 0.0532, 0.0188],
        [0.0553, 0.1154, 0.0719, 0.0945, 0.0705, 0.2141, 0.0665, 0.0610, 0.2023, 0.0487]
    ]
)

out_2 = torch.Tensor(
    [
        [0.0942, 0.0448, 0.0929, 0.0316, 0.0296, 0.0272, 0.4189, 0.0804, 0.0988, 0.0816],
        [0.2645, 0.0424, 0.0199, 0.1344, 0.0226, 0.1131, 0.2144, 0.1160, 0.0421, 0.0306]
    ]
)

out_3 = torch.Tensor(
    [
        [0.1159, 0.1263, 0.1011, 0.0870, 0.1503, 0.0259, 0.0609, 0.1611, 0.0082, 0.1633],
        [0.1230, 0.0577, 0.3204, 0.0145, 0.0699, 0.0975, 0.0466, 0.1191, 0.0841, 0.0671]
    ]
)

In [ ]:
# TO DO:
#  1. Simulate predicted probability distributions by passing them through softmax
#  2. Stack the tensors to get a tensor of shape 2x3x10 (`torch.Size([2, 3, 10])`)

<span style="color:red;">**Question 6-1:** What are the probability values for the first instance (first row) by the the third hypothetical model?</span>

**Answer**: 

<span style="color:red;">**Question 6-1:** What are the probability values for the second instance (second row) by the the second hypothetical model?</span>

**Answer**: 

<span style="color:red;">**Question 6-1:** What are the probability values for the first instance (first row) by the the first hypothetical model?</span>

**Answer**:

Other useful functions that you may explore,
* torch.nn.BCELoss is the binary cross entropy
* torch.nn.BCEWithLogitsLoss BCE without logistic function.
* torch.nn.CrossEntropyLoss is the softmax cross entropy loss
* torch.nn.MSELoss is the Mean Squared Error 
* torch.nn.NLLLoss is the negative log likelihood (log likelihood without softmax computation)